In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=50
Nul=100
Nup=10**(40)
scal=1
umax=7.5
vmax=100
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 375000000.0,75000.0X5000
Total number of points is 375000000
Runtime: about 80.13 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)




Using Standard Coordinates Along V


In [ ]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)
drunp=np.array([0.0],dtype=datatype)
dsigunp=np.array([0.0],dtype=datatype)
dphinpuf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigunpvalue=100*np.exp(np.nanmax(signpf))
drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigunp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=M0
    
    
    

    
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
    
            
            
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    print(TempTol)
    print(exterm1)
    print(exterm2)
    print(exterm3)
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.000200000000000000009584
1
-1.472174213583699
0.15041650352491942
0.4452099059837615
---
0.000200000000000000009584
0.010000000000000000208167
1
-1.4755316459570622
0.15040815764368887
0.44675624190430063
---
0.010200000000000000732747
0.010000000000000000208167
1
-1.4821610447526723
0.1503917415064393
0.4498097414418703
---
0.020200000000000002675637
0.010000000000000000208167
1
-1.488865946707918
0.15037522340968223
0.4528976758596448
---
0.030200000000000004618528
0.010000000000000000208167
1
-1.495647625367802
0.1503586020553449
0.4560206091248576
---
0.040200000000000006561418
0.010000000000000000208167
1
-1.5025073828073943
0.15034187612137373
0.45917911756711766
---
0.050200000000000008504308
0.010000000000000000208167
1
-1.5094465504057908
0.15032504426146787
0.46237379022200636
---
0.060200000000000010447199
0.010000000000000000208167
1
-1.5164664898147606
0.15030810510374693
0.46560522917690333
---
0.070200000000000012390089
0.01000000000000000020

1
-2.205723340657349
0.14895381346029524
0.7800102828242198
---
0.694087728129097802920455
0.009067320289606042513220
1
-2.2215067169307336
0.14892797134029578
0.7871376390905755
---
0.703155048418703798596141
0.009002898729755943671438
1
-2.2374345079442084
0.14890206483753055
0.7943270316735865
---
0.712157947148459702368939
0.008938809126697670401351
1
-2.253507970798233
0.14887609371405652
0.8015789601185014
---
0.721096756275157346749438
0.008875051812182250393635
1
-2.269728372687233
0.14885005773064286
0.8088939282557142
---
0.729971808087339635306989
0.008811627083077392882937
1
-2.2860969910328195
0.14882395664690407
0.8162724442444969
---
0.738783435170417002169074
0.008748535201458946738984
1
-2.302615113548726
0.1487977902212556
0.8237150206291604
---
0.747531970371875909009418
0.008685776394986204421156
1
-2.3192840383173876
0.14877155821086951
0.8312221743753879
---
0.756217746766862153329214
0.008623350857236856265020
1
-2.33610507389316
0.1487452603718964
0.838794426944

1
-3.738599029317424
0.1469751764385574
1.460664656518506
---
1.191317009695666451207785
0.005349597494452756722694
1
-3.768173715444643
0.14694418081520055
1.4736257103505372
---
1.196666607190119302472908
0.005307610930469008198029
1
-3.7980110099814883
0.1469131008430935
1.4866972151499136
---
1.201974218120588355773748
0.005265914171243405751677
1
-3.8281131506555166
0.14688193618019874
1.4998801147336667
---
1.207240132291831802291426
0.005224505967535272334124
1
-3.858482394216135
0.14685068648270239
1.5131753618555022
---
1.212464638259367122330445
0.005183385060919288266079
1
-3.889121016831594
0.14681935140496982
1.526583918290906
---
1.217648023320286432280568
0.005142550183818575884609
1
-3.920031314053407
0.1467879305993236
1.5401067549185012
---
1.222790573504104916224833
0.005102000060126947456052
1
-3.95121560114614
0.14675642371626552
1.5537448518263086
---
1.227892573564231870619778
0.005061733405334435838274
1
-3.982676213280549
0.14672483040434336
1.5674991983806377


1
-6.6400971499459995
0.1445433383173267
2.719259978156257
---
1.482072600369590542612741
0.003012004124090661774660
1
-6.695172467015393
0.14450538348977915
2.7430157342533192
---
1.485084604493681181835996
0.002987227005507701350917
1
-6.7507291110334355
0.1444673145478692
2.7669770460825855
---
1.488071831499188979464066
0.002962642948790800839776
1
-6.80677124747474
0.14442913097836296
2.7911457638333257
---
1.491034474447979762956606
0.002938250643786486683562
1
-6.863303079169532
0.14439083226487348
2.8155237553655983
---
1.493972725091766262650594
0.002914048785154337625064
1
-6.920328846990561
0.1443524178878164
2.8401129064004715
---
1.496886773876920706527471
0.002890036072302747463536
1
-6.977852830045324
0.1443138873245431
2.8649151206961907
---
1.499776809949223466134072
0.002866211209540527309753
1
-7.035879345935488
0.14427524004920755
2.889932320255298
---
1.502643021158764025102528
0.002842572906193121650559
1
-7.0944127512299
0.1442364755327219
2.9151664454319497
---


1
-12.02629160915322
0.14153228862756606
5.042184338277161
---
1.644185251576923079142034
0.001663023037357416441043
1
-12.128436220656141
0.14148475346651423
5.086336837538884
---
1.645848274614280404293254
0.001649017205197292335050
1
-12.231475564300924
0.14143705848752575
5.13088207201519
---
1.647497291819477682750517
0.001635125696393694089484
1
-12.335417601704423
0.1413892028852448
5.175823700011766
---
1.649132417515871473767675
0.001621347622413410504219
1
-12.440270368380519
0.14134118584885336
5.221165415468743
---
1.650753765138284867575180
0.001607682100771223865365
1
-12.546041974869816
0.1412930065620266
5.266910948469337
---
1.652361447239055980418243
0.001594128254955685464536
1
-12.652740607345788
0.14124466420302184
5.313064065663736
---
1.653955575494011620563128
0.001580685214426084386416
1
-12.760374528427151
0.14119615794467855
5.359628570488886
---
1.655536260708437623634381
0.001567352114582894542782
1
-12.868952077825524
0.14114748695397417
5.406608303618959


...

1
-280.25062304086555
0.10071708259167256
132.81755270175657
---
1.828195125998472780892712
0.000071364694154787459653
1
-282.94599694018814
0.10043571868452172
134.15627948501478
---
1.828266490692627588288133
0.000070684866427807405446
1
-285.66855296636254
0.10015209184794927
135.5087311768997
---
1.828337175559055394202801
0.000070011206316976027004
1
-288.4185696369775
0.09986617529373909
136.87502763070535
---
1.828407186765372394177120
0.000069343662667675347875
1
-291.1963278847762
0.099577941822937
138.25528838623367
---
1.828476530428040147668867
0.000068682184783297892876
1
-294.00211104990115
0.09928736381838998
139.64963257239143
---
1.828545212612823434028542
0.000068026722422429781773
1
-296.8362048705104
0.09899441323693026
141.0581788308682
---
1.828613239335245888028680
0.000067377225796040111330
1
-299.69889746990935
0.09869906160153707
142.4810452109387
---
1.828680616561041860634873
0.000066733645565072725447
1
-302.5904793458612
0.09840127999354298
143.91834907227

1
-551.4966017755
0.07359898234822193
263.7148640772165
---
1.831844823932999410942557
0.000036264956004464160642
1
-556.845910134614
0.07306290436113194
266.0828940673961
---
1.831881088889003805775246
0.000035916578780591432965
1
-562.2430541669452
0.07252111055966459
268.45620923289744
---
1.831917005467784465011505
0.000035571804492335191549
1
-567.6881270590368
0.07197351661371032
270.8338023476892
---
1.831952577272276894149172
0.000035230611750877954651
1
-573.1812007800867
0.07142003680087328
273.21459757320895
---
1.831987807884027708027475
0.000034892979694345265603
1
-578.7223246389825
0.07086058399118356
275.59744641726985
---
1.832022700863721986763721
0.000034558887999484664405
1
-584.3115237705723
0.07029506963271981
277.9811234621983
---
1.832057259751721511875644
0.000034228316893255253297
1
-589.9487975226389
0.0697234037380201
280.36432181893736
---
1.832091488068614770767795
0.000033901247165831393995
1
-595.6341177559602
0.0691454948714143
282.7456482979176
---
1.8

1
-986.6864845380492
0.01664774160551885
202.8150193384259
---
1.833729377137297555933060
0.000020269863136276443388
1
-990.6072023704412
0.015474565507528348
187.54539347301613
---
1.833749647000433746768522
0.000020189637176210359750
1
-994.2074594205586
0.01429210377645318
171.01058736347574
---
1.833769836637610062268777
0.000020116525791967350244
1
-997.4611100218704
0.013100840823698778
153.12114320988985
---
1.833789953163402097757739
0.000020050907046953920138
1
-1000.3401929104657
0.011901331732694365
133.78102206281494
---
1.833810004070449029001111
0.000019993198455627863457
1
-1002.814802966551
0.01069420987418468
112.88710608026813
---
1.833829997268904721252625
0.000019943861958195586715
1
-1004.8529542300665
0.009480195345529241
90.32866673577952
---
1.833849941130862859495210
0.000019903409663878932946
1
-1006.4204337765209
0.0082601043269237
65.9867983466806
---
1.833869844540526816700776
0.000019872410504376810737
1
-1007.4806461499161
0.007034859458315834
39.73381661

1
-355.6181733209975
-0.015074127102754353
-3118.4775296166526
---
1.834684534108085962600398
0.000006413385958390585070
1
-348.5279906194167
-0.01491178723581621
-3139.543535930727
---
1.834690947494044266363744
0.000006370352814384827570
1
-341.5811968670284
-0.01474908684453713
-3159.8424873760146
---
1.834697317846858632961471
0.000006329429419315242802
1
-334.77477737260637
-0.014586165364310899
-3179.3916544024482
---
1.834703647276277882482987
0.000006290511573906394400
1
-328.10578844652616
-0.014423152581527487
-3198.2078753079472
---
1.834709937787851741219924
0.000006253502204910383687
1
-321.5713551388354
-0.014260169297461434
-3216.3075736759934
---
1.834716191290056697127397
0.000006218310762220271876
1
-315.1686691264828
-0.01409732793879459
-3233.706774732017
---
1.834722409600819004182881
0.000006184852676278118745
1
-308.8949866518054
-0.013934733119858045
-3250.4211202734523
---
1.834728594453495231775264
0.000006153048869654598421
1
-302.74762661666466
-0.0137724821

1
-89.28991163681613
-0.006010082561166974
-3419.524955941092
---
1.835088736321990987576669
0.000005848765620280661007
1
-87.52475871043862
-0.0059227809787759655
-3413.722861395131
---
1.835094585087611251950079
0.000005858706407065023721
1
-85.79458109792253
-0.005836639512341035
-3407.7729039332958
---
1.835100443794018376664212
0.000005868935684333818328
1
-84.09868258698131
-0.005751646762075513
-3401.67907254554
---
1.835106312729702704444890
0.000005879449405270800193
1
-82.43638097864202
-0.005667791269067801
-3395.445274406223
---
1.835112192179108037137780
0.000005890243659867995352
1
-80.80700778368315
-0.00558506152526228
-3389.075336463857
---
1.835118082422767971095823
0.000005901314669761189861
1
-79.20990791135968
-0.005503445982912858
-3382.5730069935507
---
1.835123983737437791674552
0.000005912658783313626058
1
-77.64443944328974
-0.005422933063511937
-3375.941957143277
---
1.835129896396221038656904
0.000005924272470882173435
1
-76.10997330993388
-0.005343511166203

In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [ ]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

In [ ]:
#print(rnpf[1,0])

In [ ]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [ ]:
#print(rnpf)

In [ ]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [ ]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            